# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [94]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [95]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [96]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[:10])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91'], ['Manowar', 'Logged In', 'Jacob', 'M', '0', 'Klein', '247.562', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Shell Shock', '200', '1.54354E+12', '73'], ['Morcheeba', 'Logged In', 'Jacob', 'M', '1', 'Klein', '257.41016', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Women Lose Weight (Feat: Slick Rick)', '200', '1.54354E+12', '73'], ['Maroon 5', 'Logged In', 'Jacob', 'M', '2', 'Klein', '231.23546', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', "Won't Go Home Without You", '200', '1.54354E+12', '73'], ['Train', 'Logged In', 'Jacob', 'M', '3', 'Klein', '216.76363', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049'

In [116]:
# check the number of rows
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Setting up cluster + keyspace

#### Creating a Cluster

In [98]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [99]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS event_db
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [100]:
try:
    session.set_keyspace('event_db')
except Exception as e:
    print(e)

# Create tables that answer queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query 1

In [101]:
# creating helper dict that maps columns to it's index for easy querying the file

columns = [
    'artist'
    , 'first_name'
    , 'gender'
    , 'item_in_session'
    , 'last_name'
    , 'length'
    , 'level'
    , 'location'
    , 'session_id'
    , 'song'
    , 'user_id'
]

column_dict = {c:i for i,c in enumerate(columns)}

In [102]:
# Query 1
query1 = """SELECT artist, song, length from song_info where session_id = 338 and item_in_session = 4"""

# dropping table if it exists
query = "DROP TABLE IF EXISTS song_info "
try:
    session.execute(query)
except Exception as e:
    print(e)

# create statement
query = "CREATE TABLE IF NOT EXISTS song_info "
# We are creating a partition key that is session_id + item_in_session as we need to require filter on those
# the combination of those two fields is unique, so no need to add clustering columns
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY ((session_id, item_in_session)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [103]:
# Populating table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[column_dict['session_id']])
                                , int(line[column_dict['item_in_session']])
                                , line[column_dict['artist']]
                                , line[column_dict['song']]
                                , float(line[column_dict['length']])))

#### Verifying query results

In [104]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
pd.DataFrame(list(rows))

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


# Query 2

In [105]:
query2 = """SELECT artist, song, first_name, last_name from artist_user_info where user_id = 10 and session_id = 182"""

# dropping table if it exists
query = "DROP TABLE IF EXISTS artist_user_info "
try:
    session.execute(query)
except Exception as e:
    print(e)

# create statement
query = "CREATE TABLE IF NOT EXISTS artist_user_info "
# We are creating a partition key that is user_id + session_id as those are required fields
# since we need songs to be sorted by item_in_session we add the latter as a clustering column
# session_id + item_in_session is unique so our key will be unique too
query = query + \
    """(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text
    , PRIMARY KEY ((user_id, session_id), item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [106]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_user_info (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[column_dict['user_id']])
                                , int(line[column_dict['session_id']])
                                , int(line[column_dict['item_in_session']])
                                , line[column_dict['artist']]
                                , line[column_dict['song']]
                                , line[column_dict['first_name']]
                                , line[column_dict['last_name']]))

#### Verifying query results

In [107]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
pd.DataFrame(list(rows))

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [108]:
# let's check if it actually sorted by item_in_session
query2b = """SELECT artist, song, first_name, last_name, item_in_session from artist_user_info where user_id = 10 and session_id = 182"""
try:
    rows = session.execute(query2b)
except Exception as e:
    print(e)
    
pd.DataFrame(list(rows))

,artist,song,first_name,last_name,item_in_session
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0
1,Three Drives,Greece 2000,Sylvie,Cruz,1
2,Sebastien Tellier,Kilometer,Sylvie,Cruz,2
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3


# Query 3

In [109]:
query3 = """SELECT first_name, last_name from song_user_info where song = 'All Hands Against His Own'"""

# dropping table if it exists
query = "DROP TABLE IF EXISTS song_user_info "
try:
    session.execute(query)
except Exception as e:
    print(e)

# create statement
query = "CREATE TABLE IF NOT EXISTS song_user_info "
# We are creating a partition key that is song since this is a required field
# since we need users per song we are adding user_id as a clustering column
# note that this get rid of duplicates as in a user that will listen to the same song twice
# will get only one row in this table, which is fine for the purpose of the
# query we are trying to run.
query = query + \
    """(song text, user_id int, first_name text, last_name text
    , PRIMARY KEY ((song), user_id))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [110]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_info (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[column_dict['song']]
                                , int(line[column_dict['user_id']])
                                , line[column_dict['first_name']]
                                , line[column_dict['last_name']]))

#### Verifying query results

In [111]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
pd.DataFrame(list(rows))

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Dropping tables

In [113]:
for t in ['song_info', 'artist_user_info', 'song_user_info']:
    query = "drop table {}".format(t)
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [114]:
session.shutdown()
cluster.shutdown()